Import tools.

In [1]:
import pandas as pd
import numpy as np
from pickle import load
from pickle import dump
pd.set_option("max_columns", None)
pd.set_option('max_rows', None)
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from imblearn.over_sampling import SMOTE
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split, GridSearchCV,\
cross_validate, cross_val_predict, cross_val_score
from imblearn.pipeline import Pipeline as imbpipe
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import confusion_matrix, plot_confusion_matrix
from sklearn.metrics import accuracy_score, make_scorer, f1_score, precision_score, recall_score

Bring in dataframe that has race, poverty, and results joined and has Alaska all tidied up.

In [2]:
df = load(open('df_all.pkl', 'rb'))

Look at and drop county and all but one state column for modeling. Drop id.

In [3]:
df = df.drop(['County_x', 'id', 'State_x', 'County_y', 'State_y', 'county'], axis = 1)

In [4]:
state_dummies = pd.get_dummies(df['state'], drop_first = True)
central_outlying = pd.get_dummies(df['central_outlying'], drop_first = True)

In [5]:
df = df.drop(['state', 'central_outlying'], axis = 1)

In [6]:
df = pd.concat([df, state_dummies, central_outlying], axis = 1)

In [7]:
df.Target = df.Target.map({'Trump': 0, 'Clinton': 1})

In [8]:
X = df.drop(['Target'], axis = 1)
y = df.Target

In [9]:
f1_scores = make_scorer(f1_score)

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, random_state = 42)

In [11]:
pipeline = imbpipe(steps = [
    ('ss', StandardScaler(with_mean = False, with_std = False)),
    ('sm', SMOTE()),
    ('linsvc', LinearSVC(class_weight = 'balanced'))
])

In [12]:
pipeline.fit(X_train, y_train)

C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Pipeline(steps=[('ss', StandardScaler(with_mean=False, with_std=False)),
                ('sm', SMOTE()),
                ('linsvc', LinearSVC(class_weight='balanced'))])

In [13]:
results = cross_validate(pipeline, X_train, y_train, return_train_score = True, scoring = f1_scores)

C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\angie\anaconda3\envs\learn-env\lib\site-packages\sklearn\svm\_base.py:976: Converge

In [14]:
results['test_score'].mean()

0.8852391242981857

In [15]:
results['train_score'].mean()

0.8837088181917551

In [16]:
# y_hat_train = cross_val_predict(pipe2, X_train, y_train)

In [17]:
# confusion_matrix(y_train, y_hat_train)